# Project1. Rock_Scissior_Paper 분류 모델 만들기

### 1. 가위, 바위, 보 사진을 찍어서 각각의 폴더에 저장한다.

### 2. 찍은 사진의 사이즈를 변경한다.
여기서는 224x224 -> 28x28

### 3. 사진들에 해당하는 라벨을 붙여서 데이터셋을 만든다.

### 4. 똑같은 방법으로 테스트셋을 만든다.
이때 사진의 환경이나 모양이 다양할수록 좋다.

### 5. 가위바위보 분류 모델을 설계한다.
Overfitting을 줄이고 정확도를 높인다.

### 6. 테스트 데이터로 모델을 평가한다.


## 모델 설계 과정
- train 데이터의 개수가 적었을 때는 정확도가 낮은 경향이 있었다. 그래서 팀원들의 데이터를 수집해서 총 9000장의 data를 수집했다.
- 사진을 수집하는 과정에서 .jpg파일과 .png파일이 혼용되는 경우가 있었다. 그래서 모든 파일을 쓰기 위해 *.* 으로 바꿨다.
- 가위바위보 사진은 MNIST 데이터보다 추출할 특징이 많다고 생각했다. 그래서 컨볼루션 레이어를 하나 더 추가했다.
- epoch의 수를 높이면 정확도가 오히려 정확도가 떨어졌다. 그래서 Dropout과 earlystop을 사용해서 과적합을 줄였다.
- 또한 과적합을 줄이기 위해서 keras.regularizers.l2(0.001)를 사용해 가중치를 규제했다.
#### 결과적으로 데이터가 많을 수록, 과적합을 줄일 수록 더 좋은 모델이 된다. 

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from PIL import Image
import os, glob

In [2]:
def load_data(img_path):
    # 가위 : 0, 바위 : 1, 보 : 2
    number_of_data=9500# 가위바위보 이미지 개수 총합에 주의하세요.
    img_size=28
    color=3
    #이미지 데이터와 라벨(가위 : 0, 바위 : 1, 보 : 2) 데이터를 담을 행렬(matrix) 영역을 생성합니다.
    imgs=np.zeros(number_of_data*img_size*img_size*color,dtype=np.int32).reshape(number_of_data,img_size,img_size,color)
    labels=np.zeros(number_of_data,dtype=np.int32)

    idx=0
    for file in glob.iglob(img_path+'/scissor/*.*'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=0   # 가위 : 0
        idx=idx+1

    for file in glob.iglob(img_path+'/rock/*.*'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=1   # 바위 : 1
        idx=idx+1       
    
    for file in glob.iglob(img_path+'/paper/*.*'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=2   # 보 : 2
        idx=idx+1
        
    print("학습데이터(x_train)의 이미지 개수는",idx,"입니다.")
    return imgs, labels

train_dir = os.getenv("HOME") + "/aiffel/test1"
(x_train, y_train)=load_data(train_dir)
x_train_norm = x_train/255.0   # 입력은 0~1 사이의 값으로 정규화

print("x_train shape: {}".format(x_train.shape))
print("y_train shape: {}".format(y_train.shape))

학습데이터(x_train)의 이미지 개수는 9225 입니다.
x_train shape: (9500, 28, 28, 3)
y_train shape: (9500,)


In [3]:
# x_test, y_test를 만드는 방법은 x_train, y_train을 만드는 방법과 아주 유사합니다.
def load_data(ima_path):
    # 가위 : 0, 바위 : 1, 보 : 2
    number_of_data=1173 # 가위바위보 이미지 개수 총합에 주의하세요.
    img_size=28
    color=3
    #이미지 데이터와 라벨(가위 : 0, 바위 : 1, 보 : 2) 데이터를 담을 행렬(matrix) 영역을 생성합니다.
    imgs=np.zeros(number_of_data*img_size*img_size*color,dtype=np.int32).reshape(number_of_data,img_size,img_size,color)
    labels=np.zeros(number_of_data,dtype=np.int32)

    idx=0
    for file in glob.iglob(ima_path+'/scissor/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=0   # 가위 : 0
        idx=idx+1

    for file in glob.iglob(ima_path+'/rock/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=1   # 바위 : 1
        idx=idx+1       
    
    for file in glob.iglob(ima_path+'/paper/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=2   # 보 : 2
        idx=idx+1
        
    print("테스트데이터(x_test)의 이미지 개수는",idx,"입니다.")
    return imgs, labels

test_dir = os.getenv("HOME") + "/aiffel/RSP"
(x_test, y_test)=load_data(test_dir)
x_test_norm = x_test/255.0   # 입력은 0~1 사이의 값으로 정규화

print("x_test shape: {}".format(x_test.shape))
print("y_test shape: {}".format(y_test.shape))

테스트데이터(x_test)의 이미지 개수는 1171 입니다.
x_test shape: (1173, 28, 28, 3)
y_test shape: (1173,)


In [4]:
n_channel_1=32
n_channel_2=64
n_channel_3 = 128
n_dense=512

IMG_HEIGHT = 28
IMG_WIDTH = 28

In [5]:
x_train_norm, x_test_norm = x_train / 255.0, x_test / 255.0
x_train_reshaped=x_train_norm.reshape( -1, 28, 28, 3)  
x_test_reshaped=x_test_norm.reshape( -1, 28, 28, 3)

In [6]:
model=keras.models.Sequential()
model.add(keras.layers.Conv2D(n_channel_1, (3,3), activation='relu', 
                              input_shape=(IMG_HEIGHT,IMG_WIDTH,3)))
model.add(keras.layers.MaxPooling2D(2,2))
model.add(keras.layers.Conv2D(n_channel_2, (3,3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2,2)))
model.add(keras.layers.Conv2D(n_channel_3, (3,3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2,2)))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(n_dense, kernel_regularizer=keras.regularizers.l2(0.001),activation='relu'))
model.add(keras.layers.Dense(3, activation='softmax'))

model.summary()

model.compile(optimizer='adam',
            loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
             metrics=['accuracy'])



callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

history = model.fit(x_train_reshaped, y_train, batch_size=100,epochs=30, 
                    verbose=1, validation_data=(x_test_reshaped, y_test),
                    callbacks =[callback])


test_loss, test_accuracy = model.evaluate(x_test_reshaped, y_test, verbose=2)
print("test_loss: {} ".format(test_loss))
print("test_accuracy: {}".format(test_accuracy))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 128)         73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 1, 1, 128)         0         
_________________________________________________________________
dropout (Dropout)            (None, 1, 1, 128)         0